In [1]:
import pandas as pd
import numpy as np
import operator
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import feature_generator
import xgboost as xgb
from scipy import optimize
import os.path
from NN import NN
from XgBoost import XGBoostModel
from sklearn.linear_model import Ridge
from CutPoints import CutPointOptimizer

Using Theano backend.


/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 10000, True)

featureImpDf = pd.read_csv('FeatureImportance.csv')
importantFeatures = featureImpDf[featureImpDf['Importance'] > 11]
# # importantFeatures = [column for column in train.columns if column in importantFeatures['Feature'].values]
importantFeatures = importantFeatures['Feature'].values
train = train[importantFeatures]
test = test[importantFeatures]

# train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)

dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')  

Scaling...


In [ ]:
model = XGBoostModel(700, 7, 0.025, 0.65, "reg:linear", 25)
model.fit(train, labels)

In [ ]:
importance = model.bst.get_fscore()

In [ ]:
importance = sorted(importance.items(), key=operator.itemgetter(1))
for item in importance:
    print item
# print train.shape

imp = pd.DataFrame()
imp['Feature'] = [item[0] for item in importance]
imp['Importance'] = [item[1] for item in importance]
imp.to_csv(path_or_buf='FeatureImportance.csv', index=False)

In [ ]:
def WritePredictionsToFile(model, modelName):
    
    kf = KFold(len(train), 3)
    num = 1
    for train_index, test_index in kf:
        foldFile = 'fold%s.csv' % str(num)
        if os.path.isfile(foldFile):
            predictionsDF = pd.read_csv(foldFile)  
        else:
            predictionsDF = pd.DataFrame()
          
        xTrain = train.iloc[train_index].values
        yTrain = labels.iloc[train_index]      
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        
        xValidate = train.iloc[test_index].values
        yValidate = labels.iloc[test_index]
        predictions = model.predict(xValidate)
        predictionsDF[modelName] = predictions
        
        print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
        print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
#         youngTrainPredictions = [trainPredictions[i] for i in range(len(xTrain)) if i in youngInd]
#         oldTrainPredictions = [trainPredictions[i] for i in range(len(trainPredictions)) if i in oldInd]
#         youngYTrain = yTrain.iloc[youngInd] 
                        
        predictionsDF.to_csv(path_or_buf=foldFile, index=False)
   
        predictionsFile = 'testPredictions%s.csv' % str(num)
        if os.path.isfile(predictionsFile):
            testDF = pd.read_csv(predictionsFile)   
        else:
            testDF = pd.DataFrame()
            
        xTest = test.values
        testPredictions = model.predict(xTest)
        testDF[modelName] = testPredictions
        testDF.to_csv(path_or_buf=predictionsFile, index=False)
    
        num += 1

In [ ]:
WritePredictionsToFile(LogisticRegression(), 'LogisticRegression')
# WritePredictionsToFile(XGBoostModel(700, 10, 0.025, 0.65, "reg:linear"), 'XGBoostRegLin')

# WritePredictionsToFile(XGBoostModel(0.3, 1, 0, 700), 'XGBoostLinear')
# WritePredictionsToFile(RandomForestRegressor(n_estimators=10, max_depth=10), 'RandomForest')
# WritePredictionsToFile(BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=20), 'BaggingDescisionTrees_n_estimators=20')
# WritePredictionsToFile(BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0), 'BaggingDescisionTreeClassifiers_n_estimators=20')
# WritePredictionsToFile(BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10), 'BaggingLinearRegression_n_estimators=10')

In [ ]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100)
train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)
WritePredictionsToFile(NN(inputShape = train.shape[1], layers = [128, 64], dropout = [0.5, 0.5], activation='sigmoid', loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 8), 'Keras')

In [ ]:

scorer = make_scorer(quadratic_weighted_kappa.quadratic_weighted_kappa)
# print len(features)
# print len(dummyVariables)

In [ ]:
def qwk_scorer(estimator, X, Y):
    predictions = np.clip(estimator.predict(X), 1, 8)
    return quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=0)
bcf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0)
bcf.fit(X_train, y_train)
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [15, 20, 25], 'max_samples': [0.9, 1.0], 'max_features': [0.9, 1.0]}]
# clf = GridSearchCV(SVR(kernel='rbf', max_iter=1000, epsilon=0.49, tol=0.01, verbose=True), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf = GridSearchCV(BaggingClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf.fit(X_train, y_train)
# SVR(kernel='rbf', max_iter=1, , tol=0.01, verbose=True)  
#C=5, g=0.1
# TestQWK: 0.24807920235173586
#C=2, g=0.05, qwk = 0.21
#C=5, g=0.05, qwk = 0.25465662455597715
# do C 

In [ ]:
print clf.best_estimator_

In [ ]:
bcp = bcf.predict(X_test)
print quadratic_weighted_kappa.quadratic_weighted_kappa(bcp, y_test)

In [ ]:
print clf.best_params_
clfPredictions = np.clip(clf.predict(X_test), 1, 8)

# dataPoints = list()

# folds = (2, 5, 10, 20)
# for K in folds:
#     _, testQwk, trainQwk = GetBestModel(GenerateNewCombinedModel, features, K)
#     dataPoints.append((K, testQwk, trainQwk))

In [ ]:
print np.min(clfPredictions)
print np.max(clfPredictions)
print clfPredictions

In [ ]:
import theano
import theano.tensor as T
from keras import backend as K

def custom_loss(y_true, y_pred):
    
#     medianCutPoints = theano.shared(np.array([2.00711375,  3.37756859,  4.31663661,  5.32317815,  6.09457793,  6.81579768, 7.52165309]))
    x = K.abs(y_true - y_pred)
    return K.mean(K.maximum(x-K.exp(-6*x), 0.0), axis=-1)
#     return K.mean((x)*(1-K.exp(-(x))), axis=-1)
#     return K.mean(K.maximum(x-0.25, 0.0), axis=-1)

In [3]:
class CombinedModel:
    
    def __init__(self, modelsFromFile, modelsToCalculate):
        self.modelsToCalculate = modelsToCalculate 
        self.modelsFromFile = modelsFromFile
        self.stackingModel = LinearRegression()
        self.boost = [0.652626727963, 0.652281118598, 0.651128156224, 0.651189716136, 0.662062433355]
        self.keras = [0.621456525096, 0.640965818589, 0.64586691993, 0.66517675706, 0.679622912852]
        
    def fit(self, X, Y, fileName, num):     
        
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))    
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            model.fit(X, Y, num)
            predictions = model.predict(X)
            stackingData[:,i] = predictions
#             df[model] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        self.stackingModel.fit(stackingData, Y)
        predictions = self.stackingModel.predict(stackingData)
#         predictions = np.mean(stackingData, axis=1)
#         predictions = np.add(self.boost[num] * stackingTrainData[:,0], self.keras[num]*stackingTrainData[:,1]) / (self.boost[num] + self.keras[num])
        
        initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])
        cpo = CutPointOptimizer(predictions, Y)
        self.cutPoints = optimize.fmin(cpo.qwk, initialCutPoints)
        
        predictions = np.searchsorted(self.cutPoints, predictions) + 1   

        trainQwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)
        print "Train QWK: %s\n" % trainQwk
                           
    def predict(self, X, fileName, num):
        stackingData = np.ndarray((X.shape[0], len(self.modelsFromFile) + len(self.modelsToCalculate)))
        df = pd.read_csv(fileName) if os.path.isfile(fileName) else pd.DataFrame()
        
        for i in range(len(self.modelsToCalculate)):
            model = self.modelsToCalculate[i]
            predictions = model.predict(X)
            stackingData[:,i] = predictions
            
        if len(self.modelsFromFile) > 0:
            colsToChange = range(len(self.modelsToCalculate), len(self.modelsToCalculate) + len(self.modelsFromFile))
            stackingData[:,colsToChange] = df[self.modelsFromFile].values
            
        predictions = self.stackingModel.predict(stackingData)
#         predictions = np.mean(stackingData, axis=1)
#         predictions = np.add(stackingData[:,0], np.add(0.5*stackingData[:,1], 0.5*stackingData[:,2])) / 2
#         predictions = np.add(self.boost[num] * stackingData[:,0], self.keras[num]*stackingData[:,1]) / (self.boost[num] + self.keras[num])
        return np.searchsorted(self.cutPoints, predictions) + 1      
#         return np.searchsorted(np.median(cpa, axis=0), predictions) + 1      

In [4]:
from keras import objectives

# X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=1892)
X_train = train
y_train = labels

folds = 7
kf = KFold(len(X_train), folds)
num = 1
combinedModels = list()
qwks = list()

cpa = np.ndarray((5, 7))
# cpa[0,:] = [1.94094589,  2.77827061, 4.47922564,  5.59375133,  5.98938611,  6.86875996, 7.45790894]
cpa[0,:] = [2.27420049,  3.32644001,  3.22110338,  5.37172015,  6.01216561,  6.68318259, 7.4242825]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]

# nnModel = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 25], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss=custom_loss, optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

for train_index, test_index in kf:
    
    combinedModel = CombinedModel([], [XGBoostModel(800, 7, 0.025, 0.50, 25)])
#     combinedModel = CombinedModel(['XGBoost'], [])
    
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)

    xTrain = X_train.iloc[train_index].values
    yTrain = y_train.iloc[train_index]      
    xValidate = X_train.iloc[test_index].values
    yValidate = y_train.iloc[test_index]

    combinedModel.fit(xTrain, yTrain, trainFile, num-1)
    predictions = combinedModel.predict(xValidate, validateFile, num-1)
    qwk = quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
    qwks.append(qwk)
    print qwk
    
    combinedModels.append(combinedModel)
    num += 1
    
meanQwk = quadratic_weighted_kappa.mean_quadratic_weighted_kappa(qwks)
print "Overall Test Qwk: %s" % meanQwk

Will train until val error hasn't decreased in 50 rounds.
[0]	train-error:35.871544	val-error:36.342503
[1]	train-error:34.245148	val-error:34.701233
[2]	train-error:32.688091	val-error:33.131145
[3]	train-error:31.195490	val-error:31.623514
[4]	train-error:29.777054	val-error:30.196392
[5]	train-error:28.393902	val-error:28.800251
[6]	train-error:27.086897	val-error:27.480894
[7]	train-error:25.858973	val-error:26.245304
[8]	train-error:24.685301	val-error:25.059418
[9]	train-error:23.601839	val-error:23.964422
[10]	train-error:22.558767	val-error:22.909121
[11]	train-error:21.598154	val-error:21.940201
[12]	train-error:20.665844	val-error:20.998377
[13]	train-error:19.801683	val-error:20.127642
[14]	train-error:18.980227	val-error:19.300716
[15]	train-error:18.179907	val-error:18.492508
[16]	train-error:17.428270	val-error:17.733755
[17]	train-error:16.713148	val-error:17.010067
[18]	train-error:16.053881	val-error:16.347580
[19]	train-error:15.417114	val-error:15.703267
[20]	train-e

Optimization terminated successfully.
         Current function value: -0.735642
         Iterations: 152
         Function evaluations: 288
Train QWK: 0.73564227284

0.652540244684

[725]	train-error:2.363065	val-error:3.168331
Stopping. Best iteration:
[675]	train-error:2.405128	val-error:3.164099

Will train until val error hasn't decreased in 50 rounds.
[0]	train-error:35.821175	val-error:36.310623
[1]	train-error:34.198944	val-error:34.673470
[2]	train-error:32.645626	val-error:33.109875
[3]	train-error:31.154661	val-error:31.604784
[4]	train-error:29.739721	val-error:30.176023
[5]	train-error:28.360903	val-error:28.783718
[6]	train-error:27.058403	val-error:27.466080
[7]	train-error:25.833990	val-error:26.232618
[8]	train-error:24.664928	val-error:25.047832
[9]	train-error:23.581888	val-error:23.952629
[10]	train-error:22.543034	val-error:22.903086
[11]	train-error:21.583904	val-error:21.934755
[12]	train-error:20.655098	val-error:20.996437
[13]	train-error:19.792633	val-error:20.126598
[14]	train-error:18.973227	val-error:19.302717
[15]	train-error:18.174534	val-error:18.493113
[16]	train-error:17.421688	val-error:17.734926
[17]	train-error:16.708637	val-err


Optimization terminated successfully.
         Current function value: -0.731744
         Iterations: 178
         Function evaluations: 327
Train QWK: 0.731743566218

0.662378586455

[580]	train-error:2.496446	val-error:3.193547
Stopping. Best iteration:
[530]	train-error:2.538265	val-error:3.192762

Will train until val error hasn't decreased in 50 rounds.
[0]	train-error:35.765377	val-error:36.272198
[1]	train-error:34.142487	val-error:34.633926
[2]	train-error:32.592808	val-error:33.068970
[3]	train-error:31.103008	val-error:31.565414
[4]	train-error:29.688843	val-error:30.133406
[5]	train-error:28.309265	val-error:28.740450
[6]	train-error:27.007816	val-error:27.423500
[7]	train-error:25.787868	val-error:26.192188
[8]	train-error:24.618584	val-error:25.006731
[9]	train-error:23.539551	val-error:23.915152
[10]	train-error:22.504536	val-error:22.866282
[11]	train-error:21.548782	val-error:21.899332
[12]	train-error:20.622263	val-error:20.960426
[13]	train-error:19.762959	val-error:20.091448
[14]	train-error:18.942366	val-error:19.263807
[15]	train-error:18.144743	val-error:18.460279
[16]	train-error:17.393187	val-error:17.703228
[17]	train-error:16.681980	val-err

...

Stopping. Best iteration:
[525]	train-error:2.542270	val-error:3.226710

Will train until val error hasn't decreased in 50 rounds.
[0]	train-error:35.848961	val-error:36.110905
[1]	train-error:34.222305	val-error:34.480598
[2]	train-error:32.664753	val-error:32.918419
[3]	train-error:31.173008	val-error:31.422108
[4]	train-error:29.756943	val-error:29.998163
[5]	train-error:28.374077	val-error:28.610220
[6]	train-error:27.069532	val-error:27.299454
[7]	train-error:25.850031	val-error:26.073709
[8]	train-error:24.679684	val-error:24.897337
[9]	train-error:23.596554	val-error:23.808561
[10]	train-error:22.554886	val-error:22.761547
[11]	train-error:21.600235	val-error:21.801395
[12]	train-error:20.673265	val-error:20.871740
[13]	train-error:19.810972	val-error:20.004499
[14]	train-error:18.992886	val-error:19.181801
[15]	train-error:18.194286	val-error:18.380629
[16]	train-error:17.443108	val-error:17.626623
[17]	train-error:16.730282	val-error:16.908657
[18]	train-error:16.073219	val-er


Optimization terminated successfully.
         Current function value: -0.724815
         Iterations: 189
         Function evaluations: 341
Train QWK: 0.7248149527

0.6661796496

[518]	train-error:2.554312	val-error:3.225452
Stopping. Best iteration:
[468]	train-error:2.600008	val-error:3.223910

Will train until val error hasn't decreased in 50 rounds.
[0]	train-error:35.881931	val-error:36.186218
[1]	train-error:34.253922	val-error:34.553741
[2]	train-error:32.696144	val-error:32.986034
[3]	train-error:31.204172	val-error:31.487089
[4]	train-error:29.783813	val-error:30.054193
[5]	train-error:28.399124	val-error:28.663021
[6]	train-error:27.092188	val-error:27.347519
[7]	train-error:25.869572	val-error:26.116627
[8]	train-error:24.697483	val-error:24.938581
[9]	train-error:23.612617	val-error:23.846951
[10]	train-error:22.570894	val-error:22.800032
[11]	train-error:21.616585	val-error:21.841097
[12]	train-error:20.688026	val-error:20.906668
[13]	train-error:19.828291	val-error:20.040846
[14]	train-error:19.008121	val-error:19.214529
[15]	train-error:18.210123	val-error:18.410892
[16]	train-error:17.456327	val-error:17.655190
[17]	train-error:16.742052	val-err


Optimization terminated successfully.
         Current function value: -0.737032
         Iterations: 229
         Function evaluations: 397
Train QWK: 0.737032336471

0.662045790399
Overall Test Qwk: 0.659158083383


[634]	train-error:2.439055	val-error:3.203504
Stopping. Best iteration:
[584]	train-error:2.484253	val-error:3.201010



In [5]:
# validatePredictions = np.zeros(len(X_test))
cutPoints = np.ndarray((7, 7))
coef = 0
inter = 0
for i in range(1, folds + 1):
    model = combinedModels[i-1]
#     print model.cutPoints
    coef += model.stackingModel.coef_ / folds
    inter += model.stackingModel.intercept_ / folds
    cutPoints[i-1] = model.cutPoints
    
print coef
print inter
    
# print np.mean(cutPoints, axis=0)
#     validatePredictions += model.predict(X_test, 'combinedTestPredictions%s.csv' % str(i), i)
    
# validatePredictions /= folds
# validatePredictions = np.round(validatePredictions).astype(int)

# print quadratic_weighted_kappa.quadratic_weighted_kappa(validatePredictions, y_test)

[ 1.12606628]
-1.07077748492


In [6]:
testPredictions = np.zeros(len(test))
for i in range(1, folds + 1):
    model = combinedModels[i-1]
    testPredictions += model.predict(test, 'combinedTestPredictions%s.csv' % str(i), i)

    
testPredictions /= folds
predDf = pd.DataFrame()
predDf['Id'] = dfTest['Id']
predDf['Response'] = np.round(testPredictions).astype(int)
print predDf['Response'].values
predDf.to_csv(path_or_buf='XgBoost.csv', columns=['Id', 'Response'], index=False, header=['Id', 'Response'])

[3 6 7 ..., 3 1 2]


In [ ]:
kf = KFold(len(train), 5)
num = 1
qwks = list()

name = 'Keras'
model = NN(inputShape = train.shape[1], cutPointArray=cpa, layers = [250, 75, 15], dropout = [0.5, 0.5, 0.5], activation='relu', patience=5, loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 50)

# model = XGBoostModel(700, 7, 0.025, 0.50, "reg:linear", 25)

for train_index, test_index in kf:
    trainFile = 'combinedTrainPredictions%s.csv' % str(num)
    validateFile = 'combinedValidatePredictions%s.csv' % str(num)
    testFile = 'combinedTestPredictions%s.csv' % str(num)
    
    trainDF = pd.read_csv(trainFile) if os.path.isfile(trainFile) else pd.DataFrame()  
    validateDF = pd.read_csv(validateFile) if os.path.isfile(validateFile) else pd.DataFrame()  
    testDF = pd.read_csv(testFile) if os.path.isfile(testFile) else pd.DataFrame()  

    xTrain = train.iloc[train_index].values
    yTrain = labels.iloc[train_index]      
    xValidate = train.iloc[test_index].values
    yValidate = labels.iloc[test_index]
    
    model.fit(xTrain, yTrain)
    testPredictions = model.predict(test.values)
    trainPredictions = model.predict(xTrain)
    validatePredictions = model.predict(xValidate)
    
    trainDF[name] = trainPredictions
    validateDF[name] = validatePredictions
    testDF[name] = testPredictions
        
    trainDF.to_csv(path_or_buf=trainFile, index=False)
    validateDF.to_csv(path_or_buf=validateFile, index=False)
    testDF.to_csv(path_or_buf=testFile, index=False)
            
    num += 1

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

keywords = ['Medical_Keyword_' + str(i) for i in range(1, 49)]
# print dfTrain[keywords].sum(axis=1)

print dfTrain[]['Response'].mean()
print dfTrain['Response'].mean()
#     Medical_History_10
# # uniqueValues = dfTest['InsuredInfo_7'].unique()
# # for i in range(len(uniqueValues)):
# #     arr = dfTrain['InsuredInfo_7'].apply(lambda x: x == uniqueValues[i])
    

# for column in ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']:
   
#     if dfTrain[column].isnull().sum():
#         print column
#         print pearsonr(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         print dfTrain[column].median()
#         print len(dfTrain[dfTrain.Response == 8])
#         print len(dfTrain)
# #         print dfTrain[column]
# #         plt.plot(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
#         break
# # plt.show()
# # print dfTest['InsuredInfo_4'].unique()
# # print dfTrain['InsuredInfo_6'].apply(lambda x: x == 1)
# print dfTest['Medical_History_32'].isnull().sum()
            
# print dfTrain['Medical_History_32'].median()
# print dfTrain[dfTrain['Family_Hist_2'].notnull()]['Family_Hist_2'].max()
# # print dfTrain['Medical_History_32'].null().sum()
# # print dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'].median()
# # print len(pd.concat([dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32']]))
# # print pearsonr(dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Response'])

In [ ]:
neighborModel = GetBestModel(lambda: NearestNeighbors(), features)

In [ ]:
nearestNeighbors = neighborModel.kneighbors(dfTrain[features].values)

In [ ]:
predictions = np.zeros(len(dfTrain))
for i in range(len(dfTrain)):
#     predictions[i] = dfTrain.iloc[nearestNeighbors[1][i]].Response.mean()
    responses = dfTrain.iloc[nearestNeighbors[1][i]].Response
    weights = nearestNeighbors[0][i][4] - nearestNeighbors[0][i]
    predictions[i] = np.sum(responses * weights) / np.sum(weights)

In [ ]:
predictions = np.rint(predictions)
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain.Response)
print np.min(predictions)

In [ ]:
import scipy.stats as stats
lowerBound = np.mean(dfTrain['BMI']) - np.std(dfTrain['BMI'])
upperBound = np.mean(dfTrain['BMI']) + np.std(dfTrain['BMI'])
mean = np.mean(dfTrain['BMI'])
stats.normaltest(dfTrain['BMI'])

bmiScore = dfTrain['BMI'].apply(lambda x: max(0, lowerBound - x) if x < mean else max(0, x - upperBound))

model = LinearRegression(normalize=True)
features = ['Ins_Age', 'Wt', 'BMI']
X = np.ndarray((59381, 9))
X[:,0] = dfTrain['Ins_Age'].values
X[:,1] = dfTrain['BMI'].values
X[:,2] = dfTrain['Wt'].values
X[:,3] = dfTrain['Ins_Age'] * dfTrain['Wt']
X[:,4] = dfTrain['Ins_Age'] * dfTrain['BMI']
X[:,5] = dfTrain['Ins_Age'] * dfTrain['Ins_Age']
X[:,6] = dfTrain['Wt'] * dfTrain['Wt']
X[:,7] = dfTrain['Wt'] * dfTrain['BMI']
X[:,8] = dfTrain['Wt'] * dfTrain['Ins_Age']

model.fit(X, dfTrain['Response'])

In [ ]:
import theano
import theano.tensor as T
import quadratic_weighted_kappa
y1, y2 = T.vectors('y1', 'y2')
# s = 1 / (1 + T.exp(-x))
s = quadratic_weighted_kappa.quadratic_weighted_kappa(y1, y2)
# logistic = theano.function([y1, y2], s)

In [ ]:
#2 layer 200/75 cut points
cpa = np.ndarray((5, 7))
cpa[0,:] = [2.27420049,  3.32644001,  3.22110338,  5.37172015,  6.01216561,  6.68318259, 7.4242825]
cpa[1,:] = [1.93043217,  3.06875824,  4.8242938,   5.40717815,  6.17241788,  6.97729218, 7.46620894]
cpa[2,:] = [1.23315182,  3.39688586,  4.6380404,   5.37977014,  6.16224374,  6.73081194, 7.48322552]
cpa[3,:] = [2.33061803,  3.5055729,   4.52173829,  5.24184554,  6.1711789,   6.96746293, 7.7211743]
cpa[4,:] = [2.26716625,  3.59018596,  4.3780072,   5.21537676,  5.95488354,  6.72023874, 7.5133742]

print np.mean(cpa, axis=0)

In [ ]:
import theano
import theano.tensor as T

epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    y_pred
    '''Just another crossentropy'''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    return cce

In [ ]:

results, updates = theano.scan(lambda v: np.searchsorted(cutPoints, v), sequences=y_pred)
compute_elementwise = theano.function(inputs=[X, W, b_sym], outputs=[results])

result, updates = theano.scan(fn=lambda prior_result, A: np.searchsorted(cutPoints, A)
                              outputs_info=T.ones_like(A),
                              non_sequences=A,
                              n_steps=7)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
prediction = 1
for i in range(len(result)):
    if num < result[i]:
        break
    num += 1

# compiled function that returns A**k

power = theano.function(inputs=[A,k], outputs=prediction, updates=updates)

